In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
iris_dict = load_iris()

In [3]:
print((iris_dict["DESCR"]))

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [4]:
X = iris_dict['data']
y = iris_dict['target']

# Train Test Split: Shuffle vs. Stratified

- Shuffle

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=123)

In [6]:
unique, counts = np.unique(y_train, return_counts=True)
np.c_[unique, counts]

array([[ 0, 37],
       [ 1, 44],
       [ 2, 39]], dtype=int64)

- Stratified (preferred)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=123,
                                                    stratify=y)

In [8]:
unique, counts = np.unique(y_train, return_counts=True)
np.c_[unique, counts]

array([[ 0, 40],
       [ 1, 40],
       [ 2, 40]], dtype=int64)

# EDA

In [9]:
np.corrcoef([X_train[:, i] for i in range(X_train.shape[1])], y_train)

array([[ 1.        , -0.12706082,  0.85849357,  0.80184873,  0.78598684],
       [-0.12706082,  1.        , -0.45320563, -0.37919441, -0.45256929],
       [ 0.85849357, -0.45320563,  1.        ,  0.95881146,  0.95512923],
       [ 0.80184873, -0.37919441,  0.95881146,  1.        ,  0.95649697],
       [ 0.78598684, -0.45256929,  0.95512923,  0.95649697,  1.        ]])

# Multiclass Classification with Logistic Regression

## One-versus-All (OvA):

k binary classifiers.

### OvA Implementation

In [10]:
class OvALogisticClf(BaseEstimator, ClassifierMixin):
    def __init__(self, random_state=123):
        self.seed = random_state
        self.cls_model = dict()
    
    def fit(self, X, y):
        for cls in np.unique(y):
            y_ = np.where(y != cls, -1, cls)
            model = LogisticRegression(random_state=self.seed)
            model.fit(X, y_)
            self.cls_model[cls] = model
    
    def get_pred_probas(self, X):
        pred_probs = np.zeros(shape=(X.shape[0], 1))

        for cls, model in list(self.cls_model.items()):
            pred_prob = model.predict_proba(X)[:, 1]
            # print(pred_prob)
            pred_probs = np.c_[pred_probs, pred_prob]
        
        return pred_probs[:, 1:]
    
    def predict(self, X):
        pred_probs = self.get_pred_probas(X)
        
        return np.argmax(pred_probs, axis=1)
    
    def predict_proba(self, X):
        pred_probs = self.get_pred_probas(X)
        
        return np.amax(pred_probs, axis=1)

In [11]:
ovamodel = OvALogisticClf()
ovamodel.fit(X_train, y_train)
train_preds = ovamodel.predict(X_train)
train_preds

array([1, 0, 1, 0, 0, 2, 0, 0, 2, 1, 0, 0, 0, 2, 0, 2, 0, 2, 1, 0, 0, 1,
       2, 2, 1, 1, 0, 2, 1, 2, 1, 2, 2, 1, 0, 2, 1, 1, 2, 1, 0, 0, 2, 2,
       2, 2, 0, 1, 0, 0, 1, 2, 2, 1, 2, 0, 2, 2, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 2, 0, 2, 1, 1, 2, 2, 0, 2, 0, 1, 2, 2, 2, 0, 1, 0, 0, 1, 1, 2,
       1, 2, 2, 2, 2, 0, 1, 1, 1, 0, 0, 1, 2, 1, 1, 2, 0, 0, 2, 1, 2, 2,
       1, 1, 0, 1, 0, 0, 2, 2, 0, 1], dtype=int64)

Compare with sklearn default implementation under the hood.

In [12]:
model = LogisticRegression(random_state=123, multi_class="ovr")
model.fit(X_train, y_train)
sklearn_train_preds = model.predict(X_train)

In [13]:
(sklearn_train_preds == train_preds).all()

True

### Evaluation

In [14]:
from sklearn.metrics import balanced_accuracy_score, confusion_matrix

In [15]:
balanced_accuracy_score(y_test, ovamodel.predict(X_test))

0.9

In [16]:
confusion_matrix(y_test, ovamodel.predict(X_test))

array([[10,  0,  0],
       [ 0,  7,  3],
       [ 0,  0, 10]], dtype=int64)

## One-versus-One (OvO):

k*(k-1)/2 binary classifers.